## Import modules

In [42]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2 
import random
import math
import operator


## Standar Hough Transformation Function

In [43]:
def houghTransform(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # kernel = np.ones((15, 15), np.uint8)

    # opening = cv2.morphologyEx(gray, cv2.MORPH_OPEN, kernel)

    edges = cv2.Canny(gray, 50, 150, apertureSize=3)

    cv2.imwrite("edges.jpg", edges)

    # Compute the Hough Transform
    lines = cv2.HoughLines(edges, 1, np.pi / 180, 200)
    print("Found lines: {}".format(len(lines)))
    houghLines = []
    # Draw the lines
    for line in lines:
        rho, theta = line[0]
        if abs(theta-np.pi/90) < np.pi/9:
            continue
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + 1000 * (-b))
        y1 = int(y0 + 1000 * (a))
        x2 = int(x0 - 1000 * (-b))
        y2 = int(y0 - 1000 * (a))
        line_points = list(((x1,y1), (x2,y2)))
        cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)
        cv2.imwrite("StandardHough.jpg", img);
        # print("line_points: ", line_points)
        houghLines.append( line_points)
        # cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
    return houghLines

In [44]:

def determine(a, b):
    return a[0] * b[1] - a[1] * b[0]

def distancebetweenTwoPoints(p1, p2):
    '''
    calculate distance between point tuples
    '''
    x1, y1 = p1
    x2, y2 = p2
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
def checklineIntersection(line, p1, p2):
    '''
    check if a line passes throguh two points
    '''
    length1 = distancebetweenTwoPoints((line[0][0],line[0][1]),(p1, p2))    
    length2 = distancebetweenTwoPoints((line[1][0],line[1][1]),(p1, p2))
    length3 = distancebetweenTwoPoints((line[0][0],line[0][1]),(line[1][0],line[1][1]))

    if int(length1) + int(length2)  == int(length3):
        return True
    else:
        return False
def intersectionOftwoLines(line_1,line_2):
    '''
    finds the intersection point between two lines
    '''

    xVal = (line_1[0][0] - line_1[1][0], line_2[0][0] - line_2[1][0])
    yVal = (line_1[0][1] - line_1[1][1], line_2[0][1] - line_2[1][1])

    intersect = determine(xVal, yVal)

    if intersect == 0:
        return 0,0

    temp = (determine(*line_1), determine(*line_2))
    x = determine(temp, xVal) / intersect
    y = determine(temp, yVal) / intersect

    return x, y

# def intersectionofmultipleLines(lines):
#     '''
#     find all pairs of lines that intersect
#     '''
#     intersections = []
#     for i in range(len(lines)):
#         for j in range(i+1, len(lines)):
#             # print(lines[i])
#             # print(lines[j])
#             intersection = intersectionOftwoLines(lines[i], lines[j])
#             if intersection not in intersections:
#                 intersections.append(intersection)
#     return intersections

In [45]:
def twoLineVP(img, houghLines):
    '''
    function to get the Vanishing point using two lines
    '''
    countInt = {}
    
    for line1 in houghLines:
        for line2 in houghLines:
            # if both the lines are same then don't process
            if line1 == line2:
                continue
            
            # check if point the lines are intersecting
            # print(line1, line2)
            # print(intersectionOftwoLines(line1, line2))
            pointX, pointY = intersectionOftwoLines(line1, line2)
            countInt[(pointX, pointY)] = 0

            # now check for all other lines that intersect with this pair
            for lines in houghLines:
                # if lines are same then don't process
                if lines == line1 or lines == line2:
                    continue

                # check if the line is passing through the pair of points
                flag =  checklineIntersection(lines, pointX, pointY)
                if flag:
                    countInt[(pointX, pointY)] += 1

    finalInt = max(countInt.items(), key=operator.itemgetter(1))[0]
    print("Coordinates of VP : ")
    print(finalInt)
    cv2.circle(image, (int(finalInt[0]), int(finalInt[1])), 55, (0, 255, 0), 2)
    cv2.imwrite("LinePairVP.jpg", image)



## Driver Code of Rudimentary vanishing point detection with Hough Transformation
* Obtain the HoughLines from the edges of the grayScale image
* Calculate intersection point, iP = (iX, iY) of every pair of HoughLines
* For each iP, find all the other lines that pass through that iP
* Find the iP where the maximum numbers of line intersects

In [46]:
image = cv2.imread("Corridor.jpg")
houghLines = houghTransform(image)
twoLineVP(image, houghLines)


Found lines: 37
Coordinates of VP : 
(556.3757716712147, 211.86193059308837)
